## 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltd
import seaborn as sns

from gensim.models import Word2Vec

C:\Users\JAVI\Miniconda3\envs\dhds\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## 2. Cargar datos

In [2]:
movies=pd.read_csv('../Data/movies.csv')
ratings=pd.read_csv('../Data/ratings.csv')
links=pd.read_csv('../Data/links.csv')
tags=pd.read_csv('../Data/tags.csv')
imdb=pd.read_csv('../Data/tab_imdb_info.csv',index_col=0)

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
links.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [6]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


In [7]:
imdb.head()

,actors,country,director,req_imdbid,year
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995


## 3. Feature Extraction

Corregir id's de imdb:

In [8]:
imdb['imdbId']=imdb['req_imdbid'].apply(lambda x: x[2:]).astype('int64')

In [9]:
pd.merge(links,movies,on='movieId').head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [10]:
imdb.isna().sum()

actors        32
country       12
director      94
req_imdbid     0
year           0
imdbId         0
dtype: int64

In [11]:
imdb.fillna('-',inplace=True)

In [12]:
imdb['actors'].apply(lambda x: len(x.split(','))).unique()

array([4, 2, 3, 1], dtype=int64)

In [13]:
# imdb['director1'] = imdb['director'].apply(lambda x: x.split(',')[0])
# imdb['actor1'] = imdb['actors'].apply(lambda x: x.split(',')[0])
# #imdb['actor2'] = imdb['actors'].apply(lambda x: x.split(',')[1])
# imdb['country1'] = imdb['country'].apply(lambda x: x.split(',')[0])

Creo un dataframe con todos los datos necesarios para crear features por películas:

In [14]:
mov_with_feat = pd.merge(links,imdb,on='imdbId')
mov_with_feat = pd.merge(mov_with_feat,movies,on='movieId')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862.0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


In [15]:
mov_with_feat.isna().sum()

movieId       0
imdbId        0
tmdbId        8
actors        0
country       0
director      0
req_imdbid    0
year          0
title         0
genres        0
dtype: int64

In [16]:
mov_with_feat.fillna('-',inplace=True)

In [17]:
mov_with_feat['actors'] = mov_with_feat['actors'].str.replace(' ','')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
mov_with_feat['genres'] = mov_with_feat['genres'].str.replace(' ','')
mov_with_feat['director'] = mov_with_feat['director'].str.replace(' ','')
mov_with_feat['country'] = mov_with_feat['country'].str.replace(' ','')
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy"
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy"
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance"
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance"
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy


Armado de tokens:

In [18]:
#df1 = mov_with_feat[['actor1','director1','country1']]
df1 = mov_with_feat[['actors','director','country','genres']]
df1 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)
df1 = df1.str.replace(' ','')
df1 = pd.DataFrame({'clean': df1}) 
print(df1.iloc[0].values)
sent = [row.split(',') for row in df1['clean']]
mov_sent = [a for a in zip(imdb['req_imdbid'],sent)]
mov_sent[0:2]

['TomHanks,TimAllen,DonRickles,JimVarney,JohnLasseter,USA,Adventure,Animation,Children,Comedy,Fantasy']


[('tt0114709',
  ['TomHanks',
   'TimAllen',
   'DonRickles',
   'JimVarney',
   'JohnLasseter',
   'USA',
   'Adventure',
   'Animation',
   'Children',
   'Comedy',
   'Fantasy']),
 ('tt0113497',
  ['RobinWilliams',
   'JonathanHyde',
   'KirstenDunst',
   'BradleyPierce',
   'JoeJohnston',
   'USA',
   'Adventure',
   'Children',
   'Fantasy'])]

Entrenar modelo de word2vec de gensim:

In [19]:
model = Word2Vec(sent, min_count=1,size= 50,workers=3, window =3, sg = 1)

In [20]:
#Representation for director John Lasseter
model.wv['JohnLasseter']

array([-0.05779706, -0.01801369,  0.01012435,  0.02490549, -0.09863111,
       -0.02378695,  0.03747099, -0.10208313, -0.02946062,  0.02181053,
        0.00981032, -0.00607438, -0.01981093, -0.06191008, -0.08452686,
        0.02484289,  0.13031095, -0.03265464,  0.05967576, -0.02507416,
        0.0391211 , -0.02863384, -0.00032489,  0.07442655, -0.02112328,
       -0.04212496,  0.01649513,  0.07552388, -0.06799508, -0.03127813,
       -0.05374504,  0.03922107,  0.05951696,  0.07854552,  0.00618573,
        0.01007642,  0.02614738,  0.02008487, -0.00548574,  0.01153782,
       -0.00893883,  0.04309571, -0.03095798, -0.06758938, -0.01271949,
        0.0639147 ,  0.00441754,  0.05816582, -0.02415486, -0.01478071],
      dtype=float32)

In [21]:
#Get similarity
model.wv.similarity('TomHanks', 'USA')

0.5475702176038864

In [22]:
model.wv['Adventure']

array([ 0.38138062, -0.7551327 ,  0.00865916,  0.41625643, -0.7446956 ,
        0.48539996,  0.07793624, -0.7436322 ,  0.04032438,  0.62642705,
        0.30338326, -0.17782931,  0.2958304 , -1.089879  , -0.22094749,
        0.06189395,  0.55067945, -0.27507436, -0.23936547,  0.15248977,
        0.591896  , -0.50439066,  0.18154693,  0.56801647,  0.14558977,
        0.14396915,  0.219652  ,  0.40452808, -0.796205  , -0.11516809,
       -0.31403282,  0.21200766,  1.012986  ,  0.90413916,  0.345333  ,
        0.14025499,  0.59033966,  0.20733055,  0.28009856,  0.1895054 ,
        0.22475949, -0.21817125, -0.01644237, -0.8092293 ,  0.6645535 ,
        0.96320593,  0.6135039 ,  0.5444442 , -0.3324713 , -0.46331784],
      dtype=float32)

In [23]:
model.wv['(nogenreslisted)']

array([-0.13564421, -0.04650283,  0.00776119, -0.00724831, -0.17084603,
       -0.03765997,  0.06846732, -0.18209566, -0.05905335,  0.04976904,
        0.0229392 ,  0.00174799, -0.02271788, -0.13405584, -0.19163862,
        0.04955588,  0.24131191, -0.08678605,  0.11990955, -0.04632201,
        0.08836544, -0.05000726, -0.01903313,  0.18036197, -0.05525366,
       -0.10850971,  0.03240912,  0.16836211, -0.15027794, -0.06277625,
       -0.10844663,  0.11736957,  0.06912878,  0.18500075,  0.06473946,
        0.04426077,  0.09945782,  0.05577767, -0.01804947,  0.04746909,
        0.03400557,  0.04869014, -0.0688428 , -0.09722891, -0.06791025,
        0.17254946,  0.00348954,  0.12208806, -0.01451797, -0.01551466],
      dtype=float32)

In [24]:
imdb.head()

,actors,country,director,req_imdbid,year,imdbId
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",USA,John Lasseter,tt0114709,1995,114709
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",USA,Joe Johnston,tt0113497,1995,113497
2,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",USA,Howard Deutch,tt0113228,1995,113228
3,"Whitney Houston, Angela Bassett, Loretta Devin...",USA,Forest Whitaker,tt0114885,1995,114885
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...",USA,Charles Shyer,tt0113041,1995,113041


In [25]:
imdb.dtypes

actors        object
country       object
director      object
req_imdbid    object
year          object
imdbId         int64
dtype: object

In [26]:
def rep_col(text):
    #print(text)
    rep = []
    for item in text.split(','):
        rep.append(model.wv[item])
    return np.concatenate(rep,axis=0)

In [27]:
def items_rep(item_lst):
    a=0
    for item in item_lst:
        a+=model.wv[item]
    a/=len(item_lst)
    return a



In [28]:
mov_with_feat['actors_rep'] = mov_with_feat['actors'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['director_rep'] = mov_with_feat['director'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['country_rep'] = mov_with_feat['country'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat['genres_rep'] = mov_with_feat['genres'].apply(lambda x:x.split(',')).apply(items_rep)
mov_with_feat.head()

,movieId,imdbId,tmdbId,actors,country,director,req_imdbid,year,title,genres,actors_rep,director_rep,country_rep,genres_rep
0,1,114709,862,"TomHanks,TimAllen,DonRickles,JimVarney",USA,JohnLasseter,tt0114709,1995,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy","[-0.16031985, -0.012125978, 0.018840322, -0.00...","[-0.057797063, -0.018013693, 0.010124352, 0.02...","[0.40111426, -0.94431585, 0.30691, 0.38653317,...","[0.3466633, -0.69452226, -0.022450218, 0.43731..."
1,2,113497,8844,"RobinWilliams,JonathanHyde,KirstenDunst,Bradle...",USA,JoeJohnston,tt0113497,1995,Jumanji (1995),"Adventure,Children,Fantasy","[-0.146141, -0.0039580152, 0.020822594, -0.006...","[-0.050130606, -0.02658987, 0.012218042, 0.017...","[0.40111426, -0.94431585, 0.30691, 0.38653317,...","[0.3341992, -0.68723947, 0.0046548755, 0.46521..."
2,3,113228,15602,"WalterMatthau,JackLemmon,SophiaLoren,Ann-Margret",USA,HowardDeutch,tt0113228,1995,Grumpier Old Men (1995),"Comedy,Romance","[-0.18133293, -0.013688635, 0.016494844, -0.01...","[-0.09096009, -0.011482924, 0.016259573, -0.00...","[0.40111426, -0.94431585, 0.30691, 0.38653317,...","[0.33071277, -0.6900284, 0.00084304996, 0.2686..."
3,4,114885,31357,"WhitneyHouston,AngelaBassett,LorettaDevine,Lel...",USA,ForestWhitaker,tt0114885,1995,Waiting to Exhale (1995),"Comedy,Drama,Romance","[-0.11154778, -0.004495819, 0.014042145, -0.01...","[-0.19589122, -0.032701656, 0.022611862, -0.00...","[0.40111426, -0.94431585, 0.30691, 0.38653317,...","[0.31550333, -0.7216442, -0.0031799823, 0.2119..."
4,5,113041,11862,"SteveMartin,DianeKeaton,MartinShort,KimberlyWi...",USA,CharlesShyer,tt0113041,1995,Father of the Bride Part II (1995),Comedy,"[-0.17747661, -0.013665864, 0.0152197145, -0.0...","[-0.06990615, 0.0012784553, 0.014338127, 0.006...","[0.40111426, -0.94431585, 0.30691, 0.38653317,...","[0.3846935, -0.75861996, -0.026334323, 0.29418..."


In [29]:
# mov_with_feat['actors_rep'][0]

In [30]:
# mov_with_feat['director_rep'][0]

In [31]:
# mov_with_feat['country_rep'][0]

In [32]:
# np.concatenate([mov_with_feat['actors_rep'][0],mov_with_feat['director_rep'][0],mov_with_feat['country_rep'][0]],axis=0)

In [33]:
mov_with_feat['all_feat']=mov_with_feat.apply(lambda x: np.concatenate([x['actors_rep'],x['director_rep'],x['country_rep'],x['genres_rep']],axis=0),axis=1)

In [34]:
mov_with_feat[['movieId','all_feat']].head()

,movieId,all_feat
0,1,"[-0.16031985, -0.012125978, 0.018840322, -0.00..."
1,2,"[-0.146141, -0.0039580152, 0.020822594, -0.006..."
2,3,"[-0.18133293, -0.013688635, 0.016494844, -0.01..."
3,4,"[-0.11154778, -0.004495819, 0.014042145, -0.01..."
4,5,"[-0.17747661, -0.013665864, 0.0152197145, -0.0..."


In [35]:
a = mov_with_feat['all_feat'][0].reshape(1,-1)
for mov_feat in mov_with_feat['all_feat'][1:]:
    b = mov_feat.reshape(1,-1)
    a = np.concatenate([a,b])

a.shape

(9742, 200)

In [36]:
#temp_actor = rep_col(imdb['actors'][0])
#print(np.concatenate(temp_actor,axis=0))
#temp_director = rep_col(imdb['director'][0])
#temp_country = rep_col(imdb['country'][0])
#a = np.concatenate(temp_actor,axis=0)
##np.concatenate([temp_actor,temp_director,temp_country],axis=1)
#np.concatenate(temp_director,axis=0)

In [37]:
#def get_movie_features(fila):
#    temp_actor = rep_col(fila['actors'])
#    a = np.concatenate(temp_actor,axis=0)
#    temp_director = rep_col(fila['director'])
#    b = np.concatenate(temp_director,axis=0)
#    temp_country = rep_col(fila['country'])
#    c = np.concatenate(temp_country,axis=0)
#    feat_lst = [a,b,c]
#    return np.concatenate(feat_lst)
#get_movie_features(imdb.iloc[0])

#imdb['actors'].apply(rep_col)

In [38]:
# imdb['feat_str'] = imdb['actor1'] + ', ' + imdb['director1'] + ', ' + imdb['country1']
# imdb['feat_str'] = imdb['feat_str'].str.replace(' ','')
# imdb.head()

In [39]:
# imdb['feat_str'][0]

In [40]:
# imdb['movie_feat'] = imdb['feat_str'].apply(rep_col)
# imdb.head(5)

In [41]:
# mov_with_feat['genres'] = mov_with_feat['genres'].str.replace('|',',')
# mov_with_feat['feat_str'] = mov_with_feat['actor1'] + ', ' + mov_with_feat['director1'] + ', ' + mov_with_feat['country1'] #+ ', ' + mov_with_feat['genres']  
# mov_with_feat['feat_str'] = mov_with_feat['feat_str'].str.replace(' ','')
# mov_with_feat.head()

In [42]:
# mov_with_feat['features'] = mov_with_feat['feat_str'].apply(rep_col)
# mov_with_feat[['movieId','features']].head()

In [43]:
# a = mov_with_feat['features'][0].reshape(1,-1)
# for mov_feat in mov_with_feat['features'][1:]:
#     b = mov_feat.reshape(1,-1)
#     a = np.concatenate([a,b])

# a.shape

In [44]:
# np.median(a)

In [45]:
# np.mean(a)

In [46]:
# lista = [sublista.split('|') for sublista in movies['genres']]
# genres = sorted(set([item for s in lista for item in s]))
# dummies = pd.DataFrame(np.zeros((len(movies), len(genres)), dtype=int), columns=genres)
# for i, gen in enumerate(movies['genres']):
#     dummies.loc[i, gen.split('|')] = 0.2
# dummies.values

In [47]:
# dummies.shape

## 4. Obtención de peliculas similares

In [48]:
from sklearn.metrics.pairwise import cosine_similarity

#movie_features = np.concatenate([a,dummies],axis=1)
movie_features = a
cosine_sim = cosine_similarity(movie_features,dense_output=True)
cosine_sim[0:5]

array([[1.0000001 , 0.9990484 , 0.98829484, ..., 0.961938  , 0.9475195 ,
        0.98423564],
       [0.9990484 , 1.        , 0.9843519 , ..., 0.95494556, 0.94320107,
        0.979211  ],
       [0.98829484, 0.9843519 , 1.0000002 , ..., 0.9864166 , 0.9299316 ,
        0.99008983],
       [0.97758496, 0.9726308 , 0.9938849 , ..., 0.9925827 , 0.9278924 ,
        0.98703516],
       [0.9875279 , 0.9813743 , 0.9941152 , ..., 0.987648  , 0.9503884 ,
        0.99669945]], dtype=float32)

In [49]:
# movies.loc[movies['movieId'] == 1,'title'].values[0]

In [50]:
results = {}

for idx, row in movies.iterrows():
    similar_indices = cosine_sim[idx].argsort()[:-100:-1]
    similar_items = [(cosine_sim[idx][i], movies['movieId'][i]) for i in similar_indices]
    results[row['movieId']] = similar_items[1:]

## 5. Recomendar películas similares a una película en particular

In [51]:
def item(mid):
    return movies.loc[movies['movieId'] == mid]['title'].tolist()[0].split(' - ')[0]

def recommend_movie(item_id, num):
    print("Recomendando " + str(num) + " productos similares a " + item(item_id) + "...")
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        
#Ingresando el ID de la película y el número de recomendaciones devuelve los títulos mejor posisionados

In [52]:
recommend_movie(1,10)

Recomendando 10 productos similares a Toy Story (1995)...
-------
Recomendado: Antz (1998) (score:0.9997197)
Recomendado: Emperor's New Groove, The (2000) (score:0.9996582)
Recomendado: Ice Age 2: The Meltdown (2006) (score:0.99960214)
Recomendado: Madagascar (2005) (score:0.99959165)
Recomendado: Monsters, Inc. (2001) (score:0.9995839)
Recomendado: The Boss Baby (2017) (score:0.9995303)
Recomendado: Ice Age (2002) (score:0.9995179)
Recomendado: Addams Family, The (1991) (score:0.9994908)
Recomendado: Horton Hears a Who! (2008) (score:0.9994749)
Recomendado: Cool World (1992) (score:0.99946725)


In [53]:
recommend_movie(648,10)

Recomendando 10 productos similares a Mission: Impossible (1996)...
-------
Recomendado: Death Proof (2007) (score:0.9995389)
Recomendado: Alien³ (a.k.a. Alien 3) (1992) (score:0.9994453)
Recomendado: Blade Runner (1982) (score:0.9993215)
Recomendado: Matrix, The (1999) (score:0.99928856)
Recomendado: Scanner Darkly, A (2006) (score:0.9992103)
Recomendado: Darkman (1990) (score:0.9991901)
Recomendado: End of Days (1999) (score:0.99917096)
Recomendado: One, The (2001) (score:0.99914986)
Recomendado: Chronicles of Riddick, The (2004) (score:0.999147)
Recomendado: Island, The (2005) (score:0.99914104)


In [54]:
recommend_movie(2492,10)

Recomendando 10 productos similares a 20 Dates (1998)...
-------
Recomendado: Drive Me Crazy (1999) (score:0.99990344)
Recomendado: Boys and Girls (2000) (score:0.9998874)
Recomendado: Sweetest Thing, The (2002) (score:0.99988365)
Recomendado: Lost & Found (1999) (score:0.9998821)
Recomendado: Booty Call (1997) (score:0.99987364)
Recomendado: Sixteen Candles (1984) (score:0.99987316)
Recomendado: Amy's O (a.k.a. Amy's Orgasm) (2001) (score:0.9998625)
Recomendado: Admission (2013) (score:0.9998502)
Recomendado: Whatever It Takes (2000) (score:0.9998469)
Recomendado: Rewrite, The (2014) (score:0.9998394)


In [55]:
# imdb['actors'].apply(lambda x: len(x.split(','))).unique()

### Lista de películas vistas por usuario

In [56]:
ratings_sorted = ratings.sort_values(by='timestamp')
movies_by_user = pd.DataFrame(ratings_sorted.groupby('userId')['movieId'].apply(list))
movies_by_user.head()

,movieId
userId,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744..."
2,"[318, 79132, 131724, 115713, 99114, 112552, 35..."
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3..."
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5..."
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ..."


Para cada usuario me quedo con la última película que vio:

In [57]:
movies_by_user['movieId'][1][-1]

2492

In [58]:
movies_by_user['last_watched'] = movies_by_user['movieId'].apply(lambda x: x[-1])
movies_by_user.head()

,movieId,last_watched
userId,,
1,"[804, 1210, 2628, 2826, 2018, 3578, 3617, 3744...",2492
2,"[318, 79132, 131724, 115713, 99114, 112552, 35...",80489
3,"[1275, 2105, 1371, 914, 1263, 2080, 31, 720, 3...",2424
4,"[171, 902, 2351, 417, 1304, 2959, 2712, 162, 5...",4246
5,"[590, 592, 150, 380, 296, 457, 588, 153, 344, ...",474


Y le recomiendo las 10 películas más parecidas a esa, sin recomendarle alguna que ya vio:

In [64]:
def user_recommendation(uid=1,num=1):
    last_watched_id = movies_by_user.loc[uid]['last_watched']
    movies_watched = movies_by_user.loc[uid]['movieId']
    recommendation = results[last_watched_id]#[:num]
    i=0
    j=0
    recs=[]
    while i<num:
        if recommendation[j][1] not in movies_watched:
            recs.append(recommendation[j])
            j+=1
            i+=1
        else:
            j+=1
    
    #print("Recomendando " + str(num) + " productos similares a " + item(last_watched_id) + "...")
    print("Porque ya viste " + item(last_watched_id) + "...")
    print("-------")
    for rec in recs:
        #print("Recomendado: " + str(rec[1]) + ' ' + item(rec[1]) + " (score:" + str(rec[0]) + ")")
        print("Recomendado: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")
    #return recommendation

In [65]:
uid=610
num=10
user_recommendation(uid,num)

Porque ya viste Hellraiser (1987)...
-------
Recomendado: Hellbound: Hellraiser II (1988) (score:0.9997528)
Recomendado: Curse of Frankenstein, The (1957) (score:0.9993524)
Recomendado: Horror of Dracula (Dracula) (1958) (score:0.9993259)
Recomendado: The Plague of the Zombies (1966) (score:0.9991412)
Recomendado: Mummy, The (1959) (score:0.9990238)
Recomendado: Awakening, The (1980) (score:0.99892455)
Recomendado: Black Mirror: White Christmas (2014) (score:0.99806404)
Recomendado: Legend of Hell House, The (1973) (score:0.9975152)
Recomendado: The Girl with All the Gifts (2016) (score:0.9971984)
Recomendado: Children of the Damned (1963) (score:0.99719554)


In [66]:
uid=1
num=10
user_recommendation(uid,num)

Porque ya viste 20 Dates (1998)...
-------
Recomendado: Drive Me Crazy (1999) (score:0.99990344)
Recomendado: Boys and Girls (2000) (score:0.9998874)
Recomendado: Sweetest Thing, The (2002) (score:0.99988365)
Recomendado: Lost & Found (1999) (score:0.9998821)
Recomendado: Booty Call (1997) (score:0.99987364)
Recomendado: Sixteen Candles (1984) (score:0.99987316)
Recomendado: Amy's O (a.k.a. Amy's Orgasm) (2001) (score:0.9998625)
Recomendado: Admission (2013) (score:0.9998502)
Recomendado: Whatever It Takes (2000) (score:0.9998469)
Recomendado: Rewrite, The (2014) (score:0.9998394)


In [67]:
uid=154
num=10
user_recommendation(uid,num)

Porque ya viste Finding Dory (2016)...
-------
Recomendado: Ice Age 4: Continental Drift (2012) (score:0.99983585)
Recomendado: Planes (2013) (score:0.99964774)
Recomendado: Sausage Party (2016) (score:0.9995589)
Recomendado: The Madagascar Penguins in a Christmas Caper (2005) (score:0.9994921)
Recomendado: Wreck-It Ralph (2012) (score:0.99948347)
Recomendado: ParaNorman (2012) (score:0.99943465)
Recomendado: Bolt (2008) (score:0.99940515)
Recomendado: Puss in Boots: The Three Diablos (2012) (score:0.99938387)
Recomendado: Mr. Peabody & Sherman (2014) (score:0.99937123)
Recomendado: Inside Out (2015) (score:0.999301)


In [63]:
# sorted(movies_by_user.loc[uid]['movieId'])